In [2]:
# Import necessary libraries
from datetime import datetime

Time_start = datetime.now()
print("\n Start time:", Time_start)

import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Define paths
ModelFolder = (r'C:\Users\invitados fc\Documents\Files JCTG\2025 Mezcales\Codes Modify - Alternatives')
ImagesSplittedFolder = (r'C:\Users\invitados fc\Documents\Files JCTG\2025 Mezcales\Images\Cuishe vs Tepeztate')
Training_Folder = (r'C:\Users\invitados fc\Documents\Files JCTG\2025 Mezcales\Images\Cuishe vs Tepeztate\train')
Validation_Folder = (r'C:\Users\invitados fc\Documents\Files JCTG\2025 Mezcales\Images\Cuishe vs Tepeztate\val')

# Load training data
train_images = []
train_labels = []
for folder in os.listdir(Training_Folder):
    folder_path = os.path.join(Training_Folder, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        img = load_img(os.path.join(folder_path, image_file), target_size=(224, 224))
        img_array = img_to_array(img)
        train_images.append(img_array)
        train_labels.append(folder)

# Load validation data
validation_images = []
validation_labels = []
for folder in os.listdir(Validation_Folder):
    folder_path = os.path.join(Validation_Folder, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        img = load_img(os.path.join(folder_path, image_file), target_size=(224, 224))
        img_array = img_to_array(img)
        validation_images.append(img_array)
        validation_labels.append(folder)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

# Reshape images to 1D arrays
train_images = train_images.reshape(-1, 224 * 224 * 3)
validation_images = validation_images.reshape(-1, 224 * 224 * 3)

# Convert labels to numerical labels
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
validation_labels = le.transform(validation_labels)

# Define hyperparameter space for SVM
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'degree': [1, 2, 3],
    'gamma': ['scale', 'auto']
}

# Perform grid search with SVM
grid_search = GridSearchCV(svm.SVC(), param_grid, cv=5)
grid_search.fit(train_images, train_labels)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Evaluate best model on validation set
best_model = grid_search.best_estimator_
predictions = best_model.predict(validation_images)
accuracy = accuracy_score(validation_labels, predictions)
print("Accuracy on Validation Set:", accuracy)
print("Classification Report on Validation Set:")
print(classification_report(validation_labels, predictions))
print("Confusion Matrix on Validation Set:")
print(confusion_matrix(validation_labels, predictions))


 Start time: 2025-04-04 15:49:44.643231
Best Parameters: {'C': 0.1, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 0.5875
Accuracy on Validation Set: 0.7317073170731707
Classification Report on Validation Set:
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        21
           1       0.71      0.75      0.73        20

    accuracy                           0.73        41
   macro avg       0.73      0.73      0.73        41
weighted avg       0.73      0.73      0.73        41

Confusion Matrix on Validation Set:
[[15  6]
 [ 5 15]]


In [3]:
import joblib

# Save the best model
best_model = grid_search.best_estimator_
joblib.dump(best_model, 'Cuishe_Tepeztate_best_svm_model.joblib')

# Save the LabelEncoder
joblib.dump(le, 'Cuishe_Tepeztate_label_encoder.joblib')

End_Time = datetime.now()

print('\n Duration All Training: {}'.format(End_Time - Time_start))


 Duration All Training: 0:27:56.660362
